In [7]:
# Import the required libraries and dependencies
import pandas as pd
import holoviews as hv
from prophet import Prophet
import hvplot.pandas
import datetime as dt
import numpy as np
import seaborn as sns
import warnings
%matplotlib inline

In [8]:
warnings.filterwarnings('ignore')

In [3]:
# Upload the "google_hourly_search_trends.csv" file into Colab, then store in a Pandas DataFrame
# Set the "Date" column as the Datetime Index.

from google.colab import files
uploaded = files.upload()

df_mercado_trends = pd.read_csv("google_hourly_search_trends.csv", index_col="Date", parse_dates=True, infer_datetime_format=True)

# Review the first and last five rows of the DataFrame
display(df_mercado_trends.head())
display(df_mercado_trends.tail())

ModuleNotFoundError: No module named 'google'

In [ ]:
# Review the data types of the DataFrame using the info function
df_mercado_trends.info()


In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Slice the DataFrame to just the month of May 2020
df_may_2020 = df_mercado_trends.loc["2020-05"]

# Use hvPlot to visualize the data for May 2020
df_may_2020.hvplot()


In [ ]:
# Calculate the sum of the total search traffic for May 2020
traffic_may_2020 = float(df_may_2020.sum())

# View the traffic_may_2020 value
print(f"The total search traffic for May 2020 is {traffic_may_2020:.02f}")


In [ ]:
# Calcluate the monhtly median search traffic across all months 
# Group the DataFrame by index year and then index month, chain the sum and then the median functions
median_monthly_traffic = float(df_mercado_trends.groupby([df_mercado_trends.index.year, df_mercado_trends.index.month]).sum().median())

# View the median_monthly_traffic value
print(f"The median monthly traffic is {median_monthly_traffic:.02f}")


In [ ]:
# Visualize the monthly search traffic across all the months 
hv.extension('bokeh')
# Group the DataFrame by index year and the index month
# and sum the search results for the month
df_mercado_monthly_traffic = df_mercado_trends.groupby([df_mercado_trends.index.year, df_mercado_trends.index.month])["Search Trends"].sum().reset_index(level=1).rename(columns={df_mercado_trends.index.name:'month'}).reset_index()
df_mercado_monthly_traffic = df_mercado_monthly_traffic.rename(columns={df_mercado_trends.index.name:'year'})
# fix the date
df_mercado_monthly_traffic["Date"] = pd.to_datetime(df_mercado_monthly_traffic[["year", "month"]].assign(DAY=1))
# set the index as Date
df_mercado_monthly_traffic = df_mercado_monthly_traffic.set_index("Date")
# Drop the unnecessary columns
df_mercado_monthly_traffic = df_mercado_monthly_traffic.drop(columns=["year", "month"], axis=1)
# Rename the column
df_mercado_monthly_traffic = df_mercado_monthly_traffic.rename(columns={"Search Trends":"Monthly Search Trends"})
# plot the data frame
df_mercado_monthly_traffic.hvplot()


##### Answer the following question: 

**Question:** Did the Google search traffic increase during the month that MercadoLibre released its financial results?

**Answer:** Compared to the monthly median value, May 2020 traffic increased. 


In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Group the hourly search data to plot (use hvPlot) the average traffic by the day of week 
df_mercado_weekly_trends = round(df_mercado_trends.groupby([df_mercado_trends.index.dayofweek]).mean(), 2)

df_mercado_weekly_trends.hvplot.bar(xlabel = "Day of the Week - Monday = 0, Sunday = 6",
                                    ylabel = "Average Daily Search Trends", 
                                    title = 'Average MercadoLibre Search Trends Per Day of the Week',
                                    hover_color = 'yellow')


In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Use hvPlot to visualize the hour of the day and day of week search traffic as a heatmap.
df_mercado_trends.hvplot.heatmap(
              x ='index.hour',
              y ='index.dayofweek',
              xlabel = 'Hour of the Day',
              ylabel = "Day of the Week - Monday = 0, Sunday = 6",
              C = 'Search Trends',
              cmap = 'blues',
              title = "Heatmap of Average Traffic per Hour of the Day and per Day of the Week"
      ).aggregate(
              function = np.mean
      )

##### Answer the following question:

**Question:** Does any day-of-week effect that you observe concentrate in just a few hours of that day?

**Answer:** Searches are concentrated around the midnight hours during most days

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Group the hourly search data to plot (use hvPlot) the average traffic by the week of the year
df_mercado_week_of_year_trends = round(df_mercado_trends.groupby([df_mercado_trends.index.weekofyear]).mean(), 2)

df_mercado_week_of_year_trends.hvplot.bar(
                                    xlabel = "Week of the Year",
                                    ylabel = "Average Weekly Search Trends", 
                                    title = 'Average MercadoLibre Search Trends Per Week of the Year',
                                    hover_color = 'yellow')

##### Answer the following question:

**Question:** Does the search traffic tend to increase during the winter holiday period (weeks 40 through 52)?

**Answer:** Holiday weeks have a little lower search trends than other weeks. 

In [ ]:
# Upload the "mercado_stock_price.csv" file into Colab, then store in a Pandas DataFrame
# Set the "date" column as the Datetime Index.
from google.colab import files
uploaded = files.upload()

df_mercado_stock = pd.read_csv("mercado_stock_price.csv", index_col="date", infer_datetime_format=True, parse_dates=True)

# View the first and last five rows of the DataFrame
display(df_mercado_stock.head())
display(df_mercado_stock.tail())


In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Use hvPlot to visualize the closing price of the df_mercado_stock DataFrame
df_mercado_stock.hvplot(ylabel = "Closing Price", title = "MercadoLibre Stock Closing Price", height = 500, width = 900)


In [ ]:
# Concatenate the df_mercado_stock DataFrame with the df_mercado_trends DataFrame
# Concatenate the DataFrame by columns (axis=1), and drop and rows with only one column of data
mercado_stock_trends_df = pd.concat([df_mercado_stock, df_mercado_trends], axis=1).dropna()

# View the first and last five rows of the DataFrame
display(mercado_stock_trends_df.head(5))
display(mercado_stock_trends_df.tail(5))

In [ ]:
# For the combined dataframe, slice to just the first half of 2020 (2020-01 through 2020-06) 
first_half_2020 = mercado_stock_trends_df.loc["2020-01":"2020-06"]

# View the first and last five rows of first_half_2020 DataFrame
display(first_half_2020.head())
display(first_half_2020.tail())


In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Use hvPlot to visualize the close and Search Trends data
# Plot each column on a separate axes using the following syntax
# `hvplot(shared_axes=False, subplots=True).cols(1)`
first_half_2020.hvplot(shared_axes=False, subplots=True).cols(1)


##### Answer the following question:

**Question:** Do both time series indicate a common trend that’s consistent with this narrative?

**Answer:** It looks like there is a correlation between monthly total search trends and monthly stock returns

In [ ]:
# Create a new column in the mercado_stock_trends_df DataFrame called Lagged Search Trends
# This column should shift the Search Trends information by one hour
mercado_stock_trends_df['Lagged Search Trends'] = mercado_stock_trends_df["Search Trends"].shift(1)
mercado_stock_trends_df


In [5]:
# Create a new column in the mercado_stock_trends_df DataFrame called Stock Volatility
# This column should calculate the standard deviation of the closing stock price return data over a 4 period rolling window
mercado_stock_trends_df['Stock Volatility'] = mercado_stock_trends_df["close"].pct_change().rolling(window=4).std()
mercado_stock_trends_df

NameError: name 'mercado_stock_trends_df' is not defined

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Use hvPlot to visualize the stock volatility
mercado_stock_trends_df["Stock Volatility"].hvplot()


In [ ]:
# Create a new column in the mercado_stock_trends_df DataFrame called Hourly Stock Return
# This column should calculate hourly return percentage of the closing price
mercado_stock_trends_df['Hourly Stock Return'] = mercado_stock_trends_df['close'].pct_change()


In [ ]:
# View the first and last five rows of the mercado_stock_trends_df DataFrame
display(mercado_stock_trends_df.head())
display(mercado_stock_trends_df.tail())

In [ ]:
# Construct correlation table of Stock Volatility, Lagged Search Trends, and Hourly Stock Return
corrMatrix = mercado_stock_trends_df[["Stock Volatility", "Lagged Search Trends", "Hourly Stock Return"]].corr()
sns.heatmap(corrMatrix, annot=True)

##### Answer the following question:


**Question:** Does a predictable relationship exist between the lagged search traffic and the stock volatility or between the lagged search traffic and the stock price returns?

**Answer:** It seems like there is weak correlation between these metrics. 

In [ ]:
# Using the df_mercado_trends DataFrame, reset the index so the date information is no longer the index
mercado_prophet_df = df_mercado_trends.reset_index()

# Label the columns ds and y so that the syntax is recognized by Prophet
mercado_prophet_df.columns = ["ds", "y"]

# Drop an NaN values from the prophet_df DataFrame
mercado_prophet_df = mercado_prophet_df.dropna()

# View the first and last five rows of the mercado_prophet_df DataFrame
display(mercado_prophet_df.head())
display(mercado_prophet_df.tail())


In [ ]:
# Call the Prophet function, store as an object
model_mercado_trends = Prophet()

In [ ]:
# Fit the time-series model.
model_mercado_trends.fit(mercado_prophet_df)


In [ ]:
# Create a future dataframe to hold predictions
# Make the prediction go out as far as 2000 hours (approx 80 days)
future_mercado_trends = model_mercado_trends.make_future_dataframe(periods=2000, freq="H")

# View the last five rows of the future_mercado_trends DataFrame
display(future_mercado_trends.tail())


In [ ]:
# Make the predictions for the trend data using the future_mercado_trends DataFrame
forecast_mercado_trends = model_mercado_trends.predict(future_mercado_trends)

# Display the first five rows of the forecast_mercado_trends DataFrame
display(forecast_mercado_trends.head())


In [ ]:
# Plot the Prophet predictions for the Mercado trends data
near_term_forecast = model_mercado_trends.plot(forecast_mercado_trends)

##### Answer the following question:

**Question:**  How's the near-term forecast for the popularity of MercadoLibre?

**Answer:** It is going to decrease before it increases. 


In [ ]:
# Set the index in the forecast_mercado_trends DataFrame to the ds datetime column
forecast_mercado_trends = forecast_mercado_trends.set_index("ds")

# View the only the yhat,yhat_lower and yhat_upper columns from the DataFrame
forecast_mercado_trends[['yhat', 'yhat_lower', 'yhat_upper']]

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# From the forecast_mercado_trends DataFrame, use hvPlot to visualize
#  the yhat, yhat_lower, and yhat_upper columns over the last 2000 hours 
forecast_mercado_trends[['yhat', 'yhat_lower', 'yhat_upper']].iloc[-2000:,:].hvplot(
                    width = 900, height = 500, 
                    xlabel = "Time",
                    ylabel = "Hourly Search",
                    title = "MercadoLibre Search Trends - Last 2000 hours of Prophet")


In [ ]:
# Reset the index in the forecast_mercado_trends DataFrame
forecast_mercado_trends = forecast_mercado_trends.reset_index()

# Use the plot_components function to visualize the forecast results 
# for the forecast_canada DataFrame 
figures_mercado_trends = model_mercado_trends.plot_components(forecast_mercado_trends)


##### Answer the following questions:

**Question:** What time of day exhibits the greatest popularity?

**Answer:** Midnight

**Question:** Which day of week gets the most search traffic? 
   
**Answer:** Tuesday

**Question:** What's the lowest point for search traffic in the calendar year?

**Answer:** Around the middle of October


In [ ]:
# Upload the "mercado_daily_revenue.csv" file into Colab, then store in a Pandas DataFrame
# Set the "date" column as the DatetimeIndex
# Sales are quoted in millions of US dollars
from google.colab import files
uploaded = files.upload()

df_mercado_sales = pd.read_csv("mercado_daily_revenue.csv", index_col="date", parse_dates=True, infer_datetime_format=True)

# Review the DataFrame
display(df_mercado_sales.head())
display(df_mercado_sales.tail())


In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Use hvPlot to visualize the daily sales figures 
df_mercado_sales.hvplot()

In [ ]:
# Apply a Facebook Prophet model to the data.

# Set up the dataframe in the neccessary format:
# Reset the index so that date becomes a column in the DataFrame
mercado_sales_prophet_df = df_mercado_sales.reset_index()

# Adjust the columns names to the Prophet syntax
mercado_sales_prophet_df.columns = ["ds", "y"]

# Visualize the DataFrame
display(mercado_sales_prophet_df.head())
display(mercado_sales_prophet_df.tail())


In [ ]:
# Create the model
mercado_sales_prophet_model = Prophet()

# Fit the model
mercado_sales_prophet_model.fit(mercado_sales_prophet_df)


In [ ]:
# Predict sales for 90 days (1 quarter) out into the future.

# Start by making a future dataframe
mercado_sales_prophet_future = mercado_sales_prophet_model.make_future_dataframe(periods=90, freq="D")

# Display the last five rows of the future DataFrame
display(mercado_sales_prophet_future.tail())


In [ ]:
# Make predictions for the sales each day over the next quarter
mercado_sales_prophet_forecast = mercado_sales_prophet_model.predict(mercado_sales_prophet_future)

# Display the first 5 rows of the resulting DataFrame
mercado_sales_prophet_forecast.head()

In [ ]:
# Use the plot_components function to analyze seasonal patterns in the company's revenue
forecast_figures = mercado_sales_prophet_model.plot_components(mercado_sales_prophet_forecast)


##### Answer the following question:

**Question:** For example, what are the peak revenue days? (Mondays? Fridays? Something else?)

**Answer:** Peak is Wednesday, but Tuesday is a close second. 

In [ ]:
# Plot the predictions for the Mercado sales
hv.extension('bokeh')
mercado_sales_prophet_forecast[["yhat", "yhat_lower", "yhat_upper"]].iloc[-90:,:].hvplot()


In [ ]:
# For the mercado_sales_prophet_forecast DataFrame, set the ds column as the DataFrame Index
mercado_sales_prophet_forecast = mercado_sales_prophet_forecast.set_index("ds")

# Display the first and last five rows of the DataFrame
display(mercado_sales_prophet_forecast.head())
display(mercado_sales_prophet_forecast.tail())

In [ ]:
# if we forecast from 2020-05-15 to 2020-08-12
# Produce a sales forecast for the finance division
# giving them a number for expected total sales next quarter.
# Provide best case (yhat_upper), worst case (yhat_lower), and most likely (yhat) scenarios.

# Create a forecast_quarter Dataframe for the period 2020-05-15 to 2020-08-12
# The DataFrame should include the columns yhat_upper, yhat_lower, and yhat
mercado_sales_forecast_quarter = mercado_sales_prophet_forecast[["yhat","yhat_lower","yhat_upper"]].loc["2020-05-15":"2020-08-12"]

# Update the column names for the forecast_quarter DataFrame
# to match what the finance division is looking for 
mercado_sales_forecast_quarter.columns = ["Most Likely", "Worst Case", "Best Case"]

# Review the last five rows of the DataFrame
display(mercado_sales_forecast_quarter.head())
display(mercado_sales_forecast_quarter.tail())



In [ ]:
# Displayed the summed values for all the rows in the forecast_quarter DataFrame
display(mercado_sales_forecast_quarter.sum())

### Based on the forecast information generated above, produce a sales forecast for the finance division, giving them a number for expected total sales next quarter. Include best and worst case scenarios, to better help the finance team plan.

**Answer:** Best case scenario is around 2340 sales while worst case is 1990. 